In [18]:
import requests
import json
import numpy as np
from hashlib import pbkdf2_hmac
import base64
import binascii
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import plotly.express as px
import re
import string
import pandas as pd

In [19]:
def getCryptoInitializers():
    metadata = requests.get('https://maps.amtrak.com/rttl/js/RoutesList.json').text
    public_key_index = 0
    for route in json.loads(metadata):
        try:
            public_key_index += route['ZoomLevel']
        except KeyError:
            pass
    
    public_key_metadata = requests.get('https://maps.amtrak.com/rttl/js/RoutesList.v.json').text
    public_key_metadata = json.loads(public_key_metadata)
    public_key = public_key_metadata['arr'][public_key_index]
    
    salt = public_key_metadata['s'][len(public_key_metadata['s'][0])]
    init_vec = public_key_metadata['v'][len(public_key_metadata['v'][0])]

    return {
        'PUBLIC_KEY': public_key,
        'CRYPTO_SALT': binascii.unhexlify(salt),
        'CRYPTO_IV': binascii.unhexlify(init_vec)
    }

In [20]:
cryptoInitializers = getCryptoInitializers()
MASTER_SEGMENT = 88

In [21]:
def decrypt(data, keyDerivationPassword):
    cipherText = base64.b64decode(data)

    key = pbkdf2_hmac('sha1', keyDerivationPassword.encode(), cryptoInitializers['CRYPTO_SALT'], 1000, 16)

    decipher = AES.new(key=key, mode=AES.MODE_CBC, iv=cryptoInitializers['CRYPTO_IV'])

    return decipher.decrypt(cipherText).decode()

In [22]:
rawData = requests.get('https://maps.amtrak.com/services/MapDataService/trains/getTrainsData').text

privateKeyCipher = rawData[-MASTER_SEGMENT:]
encryptedTrainData = rawData[:-MASTER_SEGMENT]

In [23]:
private_key = decrypt(privateKeyCipher, cryptoInitializers['PUBLIC_KEY']).split('|')[0]

In [26]:
train_data_string = decrypt(encryptedTrainData, private_key).replace('\x0c', '')
train_data = json.loads(''.join([c for c in train_data_string if c in string.printable]))

In [27]:
train_data_string[493524 - 10: 493524 + 10]

'":null}}]}'

In [28]:
len(train_data_string)

493524

In [108]:
def get_station_indicies():
    keys = train_data['features'][0]['properties'].keys()
    look_from_idx = len('Station')
    indices = [int(key[look_from_idx:]) for key in keys if 'Station' in key]
    indices.sort()
    return indices

In [109]:
def does_train_stop_at_station(train_dict, station_code):
    station_indices = get_station_indicies()
    for idx in station_indices:
        try:
            station = json.loads(train_dict['Station' + str(idx)])
        except TypeError:
            continue
        if station['code'] == station_code:
            return True
        
def get_station_info(train_dict, station_code):
    station_indices = get_station_indicies()
    for idx in station_indices:
        try:
            station = json.loads(train_dict['Station' + str(idx)])
        except TypeError:
            continue
        if station['code'] == station_code:
            return station
        
def find_denver_trains():
    trains = [train['properties'] for train in train_data['features']]
    return [train for train in trains if does_train_stop_at_station(train, 'DEN')]

In [110]:
denver_trains = find_denver_trains()

In [111]:
def parse_station_data(station_data):
    return_dict = {}
    if 'postdep' in station_data.keys():
        return_dict['status'] = 'departed'
        return_dict['actual_arrival'] = station_data['postarr']
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = None
        return_dict['actual_departure'] = station_data['postdep']
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = None
    elif 'postarr' in station_data.keys():
        return_dict['status'] = 'arrived'
        return_dict['actual_arrival'] = station_data['postarr']
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = None
        return_dict['actual_departure'] = None
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = station_data['estdep']
    else:
        return_dict['status'] = 'enroute'
        return_dict['actual_arrival'] = None
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = station_data['estarr']
        return_dict['actual_departure'] = None
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = station_data['estdep']
    return return_dict

In [112]:
def get_train_summary(train_dict):
    denver_station_summary = parse_station_data(get_station_info(train_dict, 'DEN'))
    return_dict = {
        'train_num': train_dict['TrainNum'],
        'heading': train_dict['Heading'],
        'dest': train_dict['DestCode'],
        'origin': train_dict['OrigCode'],
        'route': train_dict['RouteName'],
        'state': train_dict['TrainState'],
        'velocity': train_dict['Velocity'],
    }
    return_dict.update(denver_station_summary)
    return return_dict

In [113]:
train_df = pd.DataFrame.from_records([get_train_summary(train_dict) for train_dict in denver_trains])
for col in ['actual_arrival', 'scheduled_arrival', 'estimated_arrival', 'actual_departure', 'scheduled_departure', 'estimated_departure']:
    train_df[col] = pd.to_datetime(train_df[col]).dt.strftime('%a %b %d, %I:%M%p')

train_df

,train_num,heading,dest,origin,route,state,velocity,status,actual_arrival,scheduled_arrival,estimated_arrival,actual_departure,scheduled_departure,estimated_departure
0,6,SW,CHI,EMY,California Zephyr,Active,0.105633102357388,arrived,"Fri Jul 19, 09:31PM","Fri Jul 19, 06:27PM",NaN,NaN,"Fri Jul 19, 06:59PM","Fri Jul 19, 09:53PM"
1,5,NW,EMY,CHI,California Zephyr,Active,30.229700088501,departed,"Fri Jul 19, 07:31AM","Fri Jul 19, 07:56AM",NaN,"Fri Jul 19, 08:46AM","Fri Jul 19, 08:46AM",NaN
2,5,W,EMY,CHI,California Zephyr,Active,79.1564483642578,enroute,NaN,"Sat Jul 20, 07:56AM","Sat Jul 20, 07:56AM",NaN,"Sat Jul 20, 08:46AM","Sat Jul 20, 08:46AM"
3,6,E,CHI,EMY,California Zephyr,Active,39.6061897277832,enroute,NaN,"Sat Jul 20, 06:27PM","Sat Jul 20, 06:27PM",NaN,"Sat Jul 20, 06:59PM","Sat Jul 20, 06:59PM"


In [122]:
from amtrak import find_denver_trains, get_train_data

In [117]:
get_train_data()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 5,
   'geometry': {'type': 'Point',
    'coordinates': [-83.83098246400434, 34.279341590323426]},
   'properties': {'OBJECTID': 5,
    'lon': None,
    'lat': None,
    'gx_id': '1302773',
    'ViewStn2': None,
    'ViewStn1': None,
    'Station40': None,
    'Station39': None,
    'Station38': None,
    'Station37': None,
    'Station36': None,
    'Station35': '{"code":"NYP","tz":"E","bus":false,"scharr":"07/20/2024 17:48:00","schcmnt":"","autoarr":true,"autodep":false,"estarr":"07/20/2024 17:49:00","estarrcmnt":"01 MI LATE"}',
    'Station34': '{"code":"NWK","tz":"E","bus":false,"schdep":"07/20/2024 17:23:00","schcmnt":"","autoarr":true,"autodep":true,"estarr":"07/20/2024 17:24:00","estdep":"07/20/2024 17:26:00","estarrcmnt":"01 MI LATE","estdepcmnt":"03 MI LATE"}',
    'StatusMsg': ' ',
    'Station33': '{"code":"MET","tz":"E","bus":false,"scharr":"07/20/2024 17:08:00","schdep":"07/20/2024 17:10:00","schcmnt":

In [125]:
find_denver_trains()

[{'OBJECTID': 22,
  'lon': None,
  'lat': None,
  'gx_id': '1302467',
  'ViewStn2': None,
  'ViewStn1': None,
  'Station40': None,
  'Station39': None,
  'Station38': None,
  'Station37': None,
  'Station36': None,
  'Station35': '{"code":"CHI","tz":"C","bus":false,"scharr":"07/20/2024 14:39:00","schcmnt":"","autoarr":true,"autodep":false,"estarr":"07/20/2024 15:33:00","estarrcmnt":"54 MI LATE"}',
  'Station34': '{"code":"NPV","tz":"C","bus":false,"schdep":"07/20/2024 13:44:00","schcmnt":"","autoarr":true,"autodep":true,"estarr":"07/20/2024 14:55:00","estdep":"07/20/2024 14:56:00","estarrcmnt":"01 HR 11 MI LATE","estdepcmnt":"01 HR 12 MI LATE"}',
  'StatusMsg': ' ',
  'Station33': '{"code":"PCT","tz":"C","bus":false,"schdep":"07/20/2024 12:28:00","schcmnt":"","autoarr":true,"autodep":true,"estarr":"07/20/2024 13:44:00","estdep":"07/20/2024 13:45:00","estarrcmnt":"01 HR 16 MI LATE","estdepcmnt":"01 HR 17 MI LATE"}',
  'Station32': '{"code":"GBB","tz":"C","bus":false,"schdep":"07/20/2024

In [124]:
get_train_data()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 5,
   'geometry': {'type': 'Point',
    'coordinates': [-83.8198359152597, 34.288420571674216]},
   'properties': {'OBJECTID': 5,
    'lon': None,
    'lat': None,
    'gx_id': '1302773',
    'ViewStn2': None,
    'ViewStn1': None,
    'Station40': None,
    'Station39': None,
    'Station38': None,
    'Station37': None,
    'Station36': None,
    'Station35': '{"code":"NYP","tz":"E","bus":false,"scharr":"07/20/2024 17:48:00","schcmnt":"","autoarr":true,"autodep":false,"estarr":"07/20/2024 17:49:00","estarrcmnt":"01 MI LATE"}',
    'Station34': '{"code":"NWK","tz":"E","bus":false,"schdep":"07/20/2024 17:23:00","schcmnt":"","autoarr":true,"autodep":true,"estarr":"07/20/2024 17:24:00","estdep":"07/20/2024 17:26:00","estarrcmnt":"01 MI LATE","estdepcmnt":"03 MI LATE"}',
    'StatusMsg': ' ',
    'Station33': '{"code":"MET","tz":"E","bus":false,"scharr":"07/20/2024 17:08:00","schdep":"07/20/2024 17:10:00","schcmnt":"

In [40]:
del TrainTracker

In [1]:
from amtrak import TrainTracker

In [2]:
tt = TrainTracker()

In [3]:
tt.get_train_data()

In [4]:
tt.get_denver_train_df()

,train_num,heading,dest,origin,route,state,velocity,status,actual_arrival,scheduled_arrival,estimated_arrival,actual_departure,scheduled_departure,estimated_departure
0,6,NE,CHI,EMY,California Zephyr,Active,11.0479803085327,departed,"Fri Jul 19, 09:31PM","Fri Jul 19, 06:27PM",NaN,"Fri Jul 19, 10:21PM","Fri Jul 19, 06:59PM",NaN
1,5,N,EMY,CHI,California Zephyr,Active,14.1113300323486,departed,"Fri Jul 19, 07:31AM","Fri Jul 19, 07:56AM",NaN,"Fri Jul 19, 08:46AM","Fri Jul 19, 08:46AM",NaN
2,5,W,EMY,CHI,California Zephyr,Active,49.6413307189941,enroute,NaN,"Sat Jul 20, 07:56AM","Sat Jul 20, 07:56AM",NaN,"Sat Jul 20, 08:46AM","Sat Jul 20, 08:46AM"
3,6,E,CHI,EMY,California Zephyr,Active,69.5873336791992,enroute,NaN,"Sat Jul 20, 06:27PM","Sat Jul 20, 06:27PM",NaN,"Sat Jul 20, 06:59PM","Sat Jul 20, 06:59PM"
